<a href="https://colab.research.google.com/github/yepez26/SBBD2024_FUNK/blob/main/atribuicao_topicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import random
import IPython
import pickle

In [3]:
from google.colab import userdata
from openai import OpenAI

apikey = userdata.get('openai')

client = OpenAI(api_key = apikey)

In [4]:
# Clonar o repositório
!git clone https://github.com/yepez26/SBBD2024_FUNK
# Mudar para o diretório do repositório
%cd SBBD2024_FUNK


# Load the Excel file
file_path = 'top100_music_verses.xlsx'
data = pd.read_excel(file_path)

# Display the first few rows of the file
data.head()

Cloning into 'SBBD2024_FUNK'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 132.53 KiB | 1.38 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/SBBD2024_FUNK


,Unnamed: 0,ESTROFE,musica,artista,text_original,text,ntokens
0,0,Eu ouvi palavras ditas com carinho De que na...,"Valeu, Amigo",Pikeno e Menor,Eu ouvi palavras ditas com carinho De que na...,Eu ouvi palavras ditas com carinho De que na v...,49
1,1,Eu rezo e peço pra Deus cuidar A sua vida aben...,"Valeu, Amigo",Pikeno e Menor,Eu rezo e peço pra Deus cuidar A sua vida aben...,Eu rezo e peço pra Deus cuidar A sua vida aben...,18
2,2,"Me quis tirar do mal, eu percebi Disse verdade...","Valeu, Amigo",Pikeno e Menor,"Me quis tirar do mal, eu percebi Disse verdade...",Me quis tirar do mal eu percebi Disse verdades...,42
3,3,"Eu rezo e peço pra Deus cuidar, cuidar A sua v...","Valeu, Amigo",Pikeno e Menor,"Eu rezo e peço pra Deus cuidar, cuidar A sua v...",Eu rezo e peço pra Deus cuidar cuidar A sua vi...,41
4,4,"Eu rezo e peço pra Deus cuidar, cuidar A sua v...","Valeu, Amigo",Pikeno e Menor,"Eu rezo e peço pra Deus cuidar, cuidar A sua v...",Eu rezo e peço pra Deus cuidar cuidar A sua vi...,41


In [5]:
base_topics = pd.read_excel('info_topics.xlsx')
topics = base_topics.drop(0)
topics = topics.drop(index=[9,11]).head(14)

In [ ]:
import pickle
path = ''

all_results = []
for trecho, musica in zip(data['trecho'],data['musica']):

  print(trecho)
  for palavras,topicos,name in zip(topics['Representation'],topics['Descrição'],topics['Name']):

    prompt =        [
    {"role":"system","content":"""Considere a seguinte escala para realizar a comparação entre um tópico e um trecho:

        1: Não há nenhuma relação semântica entre o trecho e o tópico.
        2: A relação semântica é fraca entre o trecho e o tópico. Existe uma conexão mínima ou superficial entre o trecho e os temas discutidos no tópico, mas essa conexão não é particularmente evidente ou relevante.
        3: A relação semântica é moderada entre o trecho e o tópico. O trecho compartilha algumas semelhanças ou temas gerais com o tópico, indicando uma conexão discernível, mas não muito substancial.
        4: A relação semântica é forte entre o trecho e o tópico. O trecho está relacionado de forma significativa aos temas discutidos no tópico.
        5: A relação semântica é muito forte entre o trecho e o tópico. O trecho está altamente alinhado com os temas e conceitos discutidos no tópico.

      responda exclusivamente um número entre 1 e 5
    """},

    {"role": "user", "content": """dado o tópico representato pelos seguintes textos: {}

    e o seguinte trecho: "{}"
                 retorne exclusivamente um número de 1 a 5 indicando qualitativamente quanto o trecho relaciona-se com os tópicos dos textos""".format(topicos,trecho)},
     {
      "role": "assistant",
      "content":""
          }
      ]

    print(prompt)

    aux = 0
    while True:
        try:
            flag = 0
            answer = client.chat.completions.create(
                      model="gpt-3.5-turbo-0125",
                      messages=prompt,
                      temperature=0.1,
                      max_tokens=4,
                      # top_p=0.95,
                      # frequency_penalty=0,
                      # presence_penalty=0
                                        )
            print(answer.choices[0].message.content)
            flag = 1
            int(answer.choices[0].message.content)
            break  # Break out of the loop if no error occurs

            print(aux)
        except:  # Handle rate limiting errors
            print("Rate limit exceeded. Waiting for 3 seconds before retrying...")
            if flag ==1:
              aux = aux+1
              print('aux: '+str(aux))

            if aux == 5:  #n ficar infinitamente tentando um prompt
              # answer = -1
              break
            time.sleep(3)


    result = {}
    result['trecho'] = trecho
    result['topico'] = topicos
    result['score'] = answer.choices[0].message.content
    result['musica'] = musica
    result['Name'] = name

    all_results.append(result)

    time.sleep(1)
# Save the list to a file
    with open(path+'gpt35_trechos.pkl', 'wb') as file:
      pickle.dump(all_results, file)